In [1]:
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import matplotlib.pyplot as plt
import numpy as np

# Load data

In [2]:
dl_train_all, dl_val_all, dl_test_all = load_all_data(normalize=True)

# Train / Test

## Params

In [3]:
le_net = LeNet()

# Can use any network with ten input and two output nodes here
tail_net = nn.Linear(20, 2)

nb_epochs = 25
n_cv = 10 
n_cv_weight = 5

## Hyperparam opt the weight 
- Can skip and use result if using same architecture

In [4]:
weights = np.linspace(0.1, 1, 10)

In [5]:
weight_test_accuracies = np.zeros((n_cv_weight, len(weights)))

In [6]:
def hyperparam_opt(weights, n_cv_weight=5):
    for j, weight in enumerate(weights): 
    # 5 fold cross validate here 
        for i in range(n_cv_weight):
            siamese_le_net = Siamese(le_net, 
                                     softmax=False,
                                     target=tail_net, 
                                     weight_aux=weight)
            trainer = Trainer(nb_epochs=nb_epochs)
            trainer.fit(siamese_le_net, dl_train_all, dl_val_all, verbose=False)
            acc_test = trainer.test(siamese_le_net, dl_test_all, test_verbose=True, return_acc=True)
            weight_test_accuracies[i,j] = acc_test
    avg_weight_test_accuracies = np.mean(weight_test_accuracies, axis=0)
    best_weight = weights[np.argmax(avg_weight_test_accuracies)]
    return best_weight

In [7]:
best_weight = hyperparam_opt(weights, n_cv_weight=5)

loss_test=0.79	 acc_test=83.98
loss_test=1.07	 acc_test=84.47
loss_test=1.05	 acc_test=84.96
loss_test=1.34	 acc_test=85.94
loss_test=1.48	 acc_test=86.23
loss_test=1.58	 acc_test=85.74
loss_test=1.57	 acc_test=85.35
loss_test=1.56	 acc_test=85.55
loss_test=1.86	 acc_test=85.35
loss_test=1.95	 acc_test=85.74
loss_test=1.97	 acc_test=86.33
loss_test=1.89	 acc_test=85.55
loss_test=1.85	 acc_test=85.25
loss_test=1.83	 acc_test=86.23
loss_test=1.74	 acc_test=86.52
loss_test=1.73	 acc_test=85.55
loss_test=2.17	 acc_test=84.86
loss_test=2.16	 acc_test=85.64
loss_test=2.15	 acc_test=86.23
loss_test=2.32	 acc_test=86.13
loss_test=2.56	 acc_test=86.04
loss_test=2.29	 acc_test=85.74
loss_test=2.23	 acc_test=85.74
loss_test=2.12	 acc_test=86.43
loss_test=2.76	 acc_test=85.94
loss_test=3.13	 acc_test=86.33
loss_test=3.05	 acc_test=85.74
loss_test=2.73	 acc_test=85.84
loss_test=2.62	 acc_test=85.84
loss_test=3.03	 acc_test=86.82
loss_test=2.8	 acc_test=87.6
loss_test=3.13	 acc_test=87.6
loss_test=3

In [8]:
# keep best weight
best_weight

0.8

In [9]:
weight_aux = best_weight

In [10]:
test_accuracies = []

In [11]:
for i in range(n_cv):
    siamese_le_net = Siamese(le_net, weight_aux=weight_aux)
    trainer = Trainer(nb_epochs=nb_epochs)
    trainer.fit(siamese_le_net, dl_train_all, dl_val_all, verbose=False)
    acc_test = trainer.test(siamese_le_net, dl_test_all, test_verbose=True, return_acc=True)
    test_accuracies.append(acc_test)

/home/puck/coding/EE-559-Team/Project1/src/models.py:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = nn.functional.softmax(x)


loss_test=0.44	 acc_test=88.77
loss_test=0.33	 acc_test=88.38
loss_test=0.32	 acc_test=89.65
loss_test=0.32	 acc_test=90.62
loss_test=0.36	 acc_test=89.94
loss_test=0.4	 acc_test=90.04
loss_test=0.49	 acc_test=89.45
loss_test=0.51	 acc_test=89.16
loss_test=0.5	 acc_test=90.43
loss_test=0.58	 acc_test=89.65


In [12]:
avg_test_acc = np.mean(test_accuracies)
print('avg test acc', avg_test_acc)
stdev_test_acc = np.std(test_accuracies)
print('stdev test acc', stdev_test_acc)

avg test acc 89.60900000000001
stdev test acc 0.666039788601255


In [13]:
from src.utils import print_param_count

In [14]:
print_param_count(siamese_le_net)

Total number of parameters:     72268
Number of trainable parameters: 72268
